In [1]:
import torch
import models
import numpy as np
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from distutils.dir_util import copy_tree, remove_tree
import time
import os
import time
import torch
import random
from datetime import datetime

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


class MyImageFolder(ImageFolder):

    def __getitem__(self, index):
        # print(super(MyImageFolder, self).__getitem__(index))
        # print(self.imgs[index])

        return super(MyImageFolder, self).__getitem__(index) + (self.imgs[index][0],)


def load_data(data_path, tmp_path, args, shuffle=True):

    custom_transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                           transforms.Resize((256, 256)),
                                           transforms.ToTensor(),
                                           ])

    dataset = MyImageFolder(root=tmp_path,
                            transform=custom_transform)

    # print(dataset.class_to_idx)

    return DataLoader(dataset=dataset,
                      batch_size=64,
                      shuffle=shuffle,
                      num_workers=4)


In [3]:
def evaluate(dev_data, model, eval_train=False, max_input=None, use_cuda=False):
        model.eval()
        predictions = []
        actuals = []
        all_img_names = []
        good = 0
        total = 0
        nmb_of_input = 0
        for inputs, labels, image_names in dev_data:
            if max_input is not None and nmb_of_input >= max_input:
                break

            nmb_of_input += len(image_names)
            if use_cuda:
                inputs = inputs.to('cuda')
                labels = labels.to('cuda')
            pred_proba = model(inputs)

            actuals.extend(labels)
            all_img_names.extend(image_names)

            final_predicts = torch.argmax(pred_proba, dim=1)
            predictions.extend(final_predicts)
            total += len(labels)
            for a, b in zip(final_predicts, labels):
                if a == b:
                    good += 1

        if eval_train:
            return good/total
        
        acc = good / total
        return acc

In [4]:
def evaluate_details(dev_data, model, eval_train=False, max_input=None, use_cuda=False):
        model.eval()
        predictions = []
        actuals = []
        all_img_names = []
        good = 0
        total = 0
        nmb_of_input = 0
        gathered_data = dict()
        
        
        for inputs, labels, image_names in dev_data:
            if max_input is not None and nmb_of_input >= max_input:
                break

            nmb_of_input += len(image_names)
            if use_cuda:
                inputs = inputs.to('cuda')
                labels = labels.to('cuda')
            pred_proba = model(inputs)

            actuals.extend(labels.tolist())
            all_img_names.extend(image_names)

            final_predicts = torch.argmax(pred_proba, dim=1)
            predictions.extend(pred_proba.tolist())
            total += len(labels)
            for a, b in zip(final_predicts, labels):
                if a == b:
                    good += 1
        
        gathered_data["IMG"] = all_img_names
        gathered_data["LABEL"] = actuals
        gathered_data["PREDICTION"] = predictions
        
        df = pd.DataFrame(gathered_data)
#         df.to_csv("/home/petigep/college/orak/digikep2/GOLD_TEST/great_cnn_pixels_76_res.csv", index=False)
        
        
        if eval_train:
            return good/total, df
        
        acc = good / total
        return acc, df

In [49]:
PATH = "/home/petigep/college/orak/digikep2/saved_states/75percent_gold/best_model_1234.mdl"
model = torch.load(PATH)

In [68]:
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

#otthon
# generated_data_path = "/media/kiscsonti/521493CD1493B289/egyetem/mester/1.felev/digikep/logo_class/Digikep2_logo/Generator/Linux/output/"
# train_data_path = "/media/kiscsonti/521493CD1493B289/egyetem/mester/1.felev/digikep/logo_class/Digikep2_logo/Generator/Linux/train/"
# test_data_path = "/media/kiscsonti/521493CD1493B289/egyetem/mester/1.felev/digikep/logo_class/Digikep2_logo/Generator/Linux/test/"


#alagsor
test_data_path = "/home/petigep/college/orak/digikep2/GOLD_TEST/test_final"

kurvaanyad = ["fuck", "you", ]
test_data = load_data(test_data_path, test_data_path, args=None, shuffle=False)

labels_count = set()
for item in test_data.dataset.classes:
    labels_count.add(item)
print('Number of labels:', len(labels_count))


dev_acc, df = evaluate_details(test_data, model,)
print('Dev accuracy: %f' % dev_acc)






Number of labels: 7
Dev accuracy: 0.763393


In [10]:
id2label = dict()

for key, value in test_data.dataset.class_to_idx.items():
    id2label[value] = key

id2label

{0: 'adidas',
 1: 'apple',
 2: 'cocacola',
 3: 'disney',
 4: 'nike',
 5: 'nologo',
 6: 'puma'}

In [11]:
df["LABEL"] = df["LABEL"].map(id2label)
df

,IMG,LABEL,PREDICTION
0,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[0.0, -22.465499877929688, -19.48609733581543,..."
1,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-36.50383758544922, -105.61270141601562, 0.0,..."
2,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-6.333619594573975, -4.787569522857666, -7.11..."
3,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-0.6984599232673645, -9.247603416442871, -1.0..."
4,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-7.148628234863281, -238.32554626464844, -0.0..."
5,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-24.51028823852539, -173.42880249023438, 0.0,..."
6,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[0.0, -117.778564453125, -67.95711517333984, -..."
7,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-83.66567993164062, -219.59327697753906, 0.0,..."
8,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[-18.665037155151367, -73.66912078857422, 0.0,..."
9,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[0.0, -89.47723388671875, -23.587268829345703,..."


In [9]:
def fuckdabih(list_of_prediction):
    new_thing = list()
    for item in list_of_prediction:
        new_thing.append(np.power(np.e, item))
    return new_thing

In [12]:
df["PREDICTION"] = df["PREDICTION"].apply(fuckdabih)
df

,IMG,LABEL,PREDICTION
0,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[1.0, 1.7512871852844187e-10, 3.44584273857348..."
1,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[1.401473106640212e-16, 1.358271399751814e-46,..."
2,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[0.0017755951888947712, 0.008332685191243099, ..."
3,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[0.4973506724755211, 9.634226790105759e-05, 0...."
4,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[0.0007859414692874228, 3.137114432079201e-104..."
5,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[2.266298274635567e-11, 4.795435736545261e-76,..."
6,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[1.0, 7.06999931178432e-52, 3.0661957275209977..."
7,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[4.618031394225558e-37, 4.284020551698914e-96,..."
8,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[7.832084402526242e-09, 1.0136951178592445e-32..."
9,/home/petigep/college/orak/digikep2/logo/eval_...,adidas,"[1.0, 1.382073268262448e-39, 5.703997207760384..."


In [73]:
df.to_csv("/home/petigep/college/orak/digikep2/GOLD_TEST/great_cnn_pixels_76_res.csv", index=False)

### PART 2 -  Real images test


In [7]:
PATH = "/home/petigep/college/orak/digikep2/saved_states/75percent_gold/best_model_1234.mdl"
model = torch.load(PATH)

In [8]:
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

#otthon
# generated_data_path = "/media/kiscsonti/521493CD1493B289/egyetem/mester/1.felev/digikep/logo_class/Digikep2_logo/Generator/Linux/output/"
# train_data_path = "/media/kiscsonti/521493CD1493B289/egyetem/mester/1.felev/digikep/logo_class/Digikep2_logo/Generator/Linux/train/"
# test_data_path = "/media/kiscsonti/521493CD1493B289/egyetem/mester/1.felev/digikep/logo_class/Digikep2_logo/Generator/Linux/test/"


#alagsor
test_data_path = "/home/petigep/college/orak/digikep2/logo/eval_dataset/sorted/"
test_data = load_data(test_data_path, test_data_path, args=None, shuffle=False)

labels_count = set()
for item in test_data.dataset.classes:
    labels_count.add(item)
print('Number of labels:', len(labels_count))


dev_acc, df = evaluate_details(test_data, model,)
print('Dev accuracy: %f' % dev_acc)






Number of labels: 7
Dev accuracy: 0.241453


In [13]:
df.to_csv("/home/petigep/college/orak/digikep2/logo/eval_dataset/great_cnn_pixels_76_res_real_life_images.csv", index=False)